In [25]:
import pandas as pd 
import numpy as np


from sklearn.impute import SimpleImputer

### LIMPIEZA DE DATOS

##### TAMAÑO DEL DF

In [26]:
path='c:/Users/afonseca/Desktop/data_market.csv'
df=pd.read_csv(path,encoding='ISO-8859-1',delimiter=';',index_col='Row ID')

##### TAMAÑO DEL DF

In [27]:
print(f'el tamaño del dataframe es de {df.shape[0]} filas y {df.shape[1]} columnas')

el tamaño del dataframe es de 9994 filas y 20 columnas


##### TIPO COLUMNAS

In [28]:
columnas_numericas=df.select_dtypes(include=['number']).columns
columnas_categoricas=df.select_dtypes(include=['object']).columns
print(f'la cantidad de columnas numericas es {len(columnas_numericas)} y la de categoricas es {len(columnas_categoricas)}')

la cantidad de columnas numericas es 3 y la de categoricas es 17


##### CAMBIAR TIPO STRING A FLOAT

In [29]:
def remplazos(df):
    df['Profit'] = df['Profit'].str.replace('.','')
    df['Profit'] = df['Profit'].astype('Float64')
    df['Sales'] = df['Sales'].str.replace('.','')
    df['Sales'] = df['Sales'].astype('Float64')
    
    return df
    

In [30]:
df=remplazos(df)

##### VALORES NULLOS 

In [31]:
nullos=df.isnull().sum().reset_index()
nullos=nullos.rename(columns={'index':'columnas',0:'nullos_c'})
nullos=nullos[nullos.nullos_c>0]
nullos=nullos.set_index('columnas')
print(f'el numero de columnas con datos nullos es de {nullos.shape[0]}\
 para un total de {np.array(nullos.nullos_c).sum()} datos nullos en todo el data set')
nullos


el numero de columnas con datos nullos es de 4 para un total de 24 datos nullos en todo el data set


,nullos_c
columnas,
Sales,6
Quantity,6
Discount,6
Profit,6


##### FILTRAR VALORES NULLOS EN DF 

In [32]:
filtered_df = df[df.isna().any(axis=1)]

##### % VALORES NULLOS 

In [33]:
nullos['%']=round((nullos['nullos_c']/df.shape[0])*100,3)
nullos.loc['total'] = ['=',np.array(nullos['%']).sum()]
nullos

,nullos_c,%
columnas,,
Sales,6,0.06
Quantity,6,0.06
Discount,6,0.06
Profit,6,0.06
total,=,0.24


##### REMPLAZAR NULLOS POR PROMEDIO

In [34]:
#df.replace(np.nan, 0,inplace=True)
# Reemplazar TSH por la media
mean_imputer = SimpleImputer(strategy='mean')
df.loc[:,['Sales','Quantity','Discount','Profit']] \
        = mean_imputer.fit_transform(df[['Sales','Quantity','Discount','Profit']])


In [35]:
# SE VERIFICA QUE NO HAY VALORES NULLOS
df.isnull().values.any()

False

##### VALORES DUPLICADOS

In [36]:
df.index.duplicated().sum()

0

##### SELECCION DE FEATURES Y AGREGAR FUNCIONES DE TIEMPO

In [37]:
def tiempo(df):
    df['Ship Date'] = pd.to_datetime(df['Ship Date'])
    df['año']=df['Ship Date'].dt.year
    df['mes']=df['Ship Date'].dt.month
    df['dia']=df['Ship Date'].dt.day
    return df

In [38]:
# se borra pais por que solo tiene un solo country 
df__=df.loc[:,['Ship Date','Ship Mode','Customer Name','Segment','City','State','Category','Sub-Category','Quantity','Profit']]
df__=tiempo(df__)


##### VALORES UNICOS 

In [39]:
def unicos(df):
    unicos={i:df[i].nunique()  for i in df.columns }
    return pd.DataFrame(unicos,index=['unicos'])\
            .T.sort_values(by='unicos',ascending=True)

In [40]:
unicos_=unicos(df__)
unicos_

,unicos
Segment,3
Category,3
Ship Mode,4
año,5
mes,12
Quantity,15
Sub-Category,17
dia,31
State,49
City,531


Al examinar los valores únicos, se puede detectar la diversidad y la distribución de los datos en cada variable, lo que ayuda a comprender mejor las relaciones y patrones presentes

##### MEDIDAS ESTADISTICAS  PROFIT

In [57]:
promedio_profit=round(np.array(df__['Profit']).mean(),0)
max_profit=round(np.array(df__['Profit']).max(),0)
min_profit=round(np.array(df__['Profit']).min(),0)
desv_=round(np.array(df__['Profit']).std(),0)
Q1=np.percentile(np.array(df__['Profit']),25)
Q3=np.percentile(np.array(df__['Profit']),75)
IQR = Q3 - Q1
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

374340.0

##### RESUMEN ESTADISTICO

In [63]:
resumen={
    'promedio_profit':promedio_profit,
    'max_profit':max_profit,
    'min_profit':min_profit,
    'desv_':desv_,
    'Q1':Q1,
    'Q3':Q3,
    'limite_inferior':limite_inferior,
    'limite_superior':limite_superior
}
resumen=pd.DataFrame(resumen,index=['resumen'])
resumen

,promedio_profit,max_profit,min_profit,desv_,Q1,Q3,limite_inferior,limite_superior
resumen,179906.0,67199808.0,-38399904.0,1722850.0,2245.0,151083.0,-221012.0,374340.0
